## Theanets
- [main site](http://theanets.readthedocs.org/en/stable/quickstart.html)
- [git repository](https://github.com/lmjohns3/theanets)

The library implementation and its documents are both elegant and detailed. Please read the online tutorial by [Leif Johnson](http://lmjohns3.com/) if you need to learn the package. I will just try to put some notes here from my learning experience

In [1]:
import theanets
import numpy as np

In [2]:
print theanets.__version__

0.6.0pre


### code skeleton

```python
import theanets

# create a model.
net = theanets.Model(
    layers=...,
)

# train the model.
net.train(
    training_data,
    validation_data,
    algo='foo',
    # ...
)

# use the trained model.
net.predict(test_data)
```

### main steps in typical workflow for theanets
1. [Create a Network Layout](http://theanets.readthedocs.org/en/latest/creating.html)
    - error function (defines type of task, e.g., regression/classification/unsupervised)
    - layers
2. [Train the Network on Data](http://theanets.readthedocs.org/en/latest/training.html) - normalizing is important
3. [Use the network to make prediction](http://theanets.readthedocs.org/en/latest/using.html)

### usage patterns for deep learning tasks
1. Most of time you need to pick or customize a `loss` function, it defines your problem, e.g., classification (cross entropy), regression (mse) or autoencoder
    - 1.1. `loss` function composes of `error` and `regularization`, it usually involves both the output of differnet layers and the weights of them.
    - 1.2 in theanets you customize `loss` and `error` by inheritating `theanets.feedforward.Network` and overwrite the loss or err function. Using `find` method is the recommended way of getting parameters.
    - details: the `error` function only utilizes the outputs of different layers by default. And in the `loss` function, the built-in regularizaion include `l1/l2 norm for weights`, `l1/l2 norm for all hiddens`, and `contractive - Frobenius norm of hidden Jacobian`. The `setup_vars` method defines the variables needed to calculate the error.
2. The second part usually involves defining layout of `layers` in the network, e.g., their # of neurons, connections, activations, weight initializations, noise mechanism for outputs. It essentially defines a function mapping from inputs to outputs, with assistance from parameters.
    - 2.1 As common pratice, most of time you need to re-use pre-defined layers, with different number of neurons or activation functions.
    - 2.2 Sometime when you need to totally re-define the input-output mapping, you need to override the `transform` method (or `output` method if you need things other than dropout or noise), and register all the used parameters in `setup` method by calling factory method like `add_weights` or `add_bias`.
3. You also need to specify the trianing algorithm. Most of time you will choose different hyperparameters e.g., batch_size, learning_rate, momentum and etc, instead of inventing your own optimizing method (because it is hard to come up with good ones).
    - 3.1 common scinarios include (1) supervised training of the whole network (2) layer-wise unsupervised training of hidden layers and fine-turning the last one
    - 3.2 batch_size is an parameter to the trainer
    
### summary of theanets (v 0.5.3) parameters 
Some of them are hidden in the code, so if you are interested in the details, read the code! It is definitely worth the time. Most of the parameters are used as the arguments to **`theanets.Experiment` constructer** or its `train/itertrain` method. Each specific layer/trainer may have extra specific params

1. Construction of the experiment (model)
    - `network_class`: e.g., theanets.Classifier, specify the problem type, specially the `loss` and `error` function
    - `layers`: list of layers
    - `save_progress`: filename, if present, the constructor will restore the saved model
    - `weighted`: bool, whether the instance weights should be used
    - `sparse_input`: the input matrix would be a sparse (csr)
    
2. construction of the layer
    - `sparsity`: float, proportion of weights to be zero
    - `activation`: activation function, including `linear, sigmoid, logistic, tanh, softplus, softmax, relu, trel, trec, tlin, rect:max, rect:min, norm:mean, norm:max, norm:std`. Note the input layer cannot have an activation function.
3. training algorithm (can be used in sequence to approximate simulated annealing)
    - `optimize`: string for optimize method
    - `learning_rate`:
    - `momentum`: 
    - `save_progress`: filename, where the model will be saved during training
    - `save_every`:  +10 (every 10 trairning iterations), -10 (every 10 mins)
    - `validate_every`: int (default 10), validate model after this train iterations passed
    - `min_improvement`: float, relative amount of performance for patience validation
    - `patience`: int (default 10), max number of validations before performance improve
    - `weight_l1`: float, l1-norm regluarization for weights in all layers
    - `weight_l2`: float, l2-norm regluarization for weights in all layers
    - `hidden_l1`: float, l1-norm regluarization for outputs of all hidden layers
    - `hidden_l2`: float, l2-norm regluarization for outputs of all hidden layers
    - `input_noise`: std for added gaussian noise to the input layer (parameterless/activationless)
    - `input_dropout`: [0,1] as proportion for zero-outed inputs
    - `batch_size`: size for mini-batch based optimization, default 32

### [Deep Learning Tutorial](http://deeplearning.net/tutorial/) re-implemented in Theanets

In [3]:
import cPickle
from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
## MNIST data
(train_X, train_y), (valid_X, valid_y), (test_X, test_y) = cPickle.load(open("../data/mnist.pkl"))
def asint32(*args):
    return [a.astype(np.int32) for a in args]
train_y, valid_y, test_y = asint32(train_y, valid_y, test_y)
print train_X.shape, train_y.shape
print valid_X.shape, valid_y.shape
print test_X.shape, test_y.shape

(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


#### Logistic Regression - softmax/nll

In [5]:
exp = theanets.Experiment(theanets.Classifier, layers = (784, 100, 10))
for i, (train, valid) in enumerate(exp.itertrain((train_X, train_y), (valid_X, valid_y),
                                  optimize="sgd", 
                                  learning_rate=0.15, 
                                  batch_size = 300, 
                                  patience = 10, 
                                  min_improvement=0., 
                                  validate_every = 30)):
    if i % 30 == 0:
        print train["loss"], valid["loss"], "%.2f%%" % train["acc"], "%.2f%%" % valid["acc"]

1.58807266607 2.65150267426 0.64% 0.10%
0.207966851444 0.201940865977 0.94% 0.94%
0.13882959766 0.145851092205 0.96% 0.96%
0.102828069087 0.120249354151 0.97% 0.97%
0.0809079191988 0.105241694744 0.98% 0.97%
0.0657393572101 0.0962540788485 0.98% 0.97%
0.0544607075615 0.0898529635519 0.99% 0.97%
0.0457255215889 0.0851220854554 0.99% 0.97%
0.038852607498 0.0818743358044 0.99% 0.98%
0.0333096154654 0.0791988210868 0.99% 0.98%
0.028776614833 0.0775888977711 0.99% 0.98%
0.0250326388959 0.0765088036489 1.00% 0.98%
0.0219601327384 0.0755897913149 1.00% 0.98%
0.0193823026531 0.0749001912332 1.00% 0.98%
0.017192690025 0.074427059088 1.00% 0.98%
0.01535377348 0.0742420069926 1.00% 0.98%
0.013769473887 0.0742478456966 1.00% 0.98%
0.0124105442791 0.0741045716399 1.00% 0.98%
0.0112447572555 0.0744029052255 1.00% 0.98%
0.0102429089061 0.0743376309873 1.00% 0.98%
0.00936752103517 0.0744788075203 1.00% 0.98%
0.00860323026516 0.0747735993275 1.00% 0.98%
0.0079495571273 0.0748532398908 1.00% 0.98%
0.007

In [7]:
print accuracy_score(exp.network.classify(valid_X), valid_y)
print accuracy_score(exp.network.classify(test_X), test_y)

0.9781
0.9783


### theanets with hyperopt